```
입력: 훈련 데이터셋 D = {(x1, y1), (x2, y2), ..., (xn, yn)}, 여기서 xi는 특성 벡터이고 yi는 해당하는 레이블(1 또는 -1)입니다.
       반복 횟수 T

각 데이터 포인트의 가중치 초기화:
w_i = 1 / n, i = 1, 2, ..., n

t = 1부터 T까지 반복:
    가중치 w_i를 사용하여 가중치 훈련 데이터 D로 약한 학습기 h_t를 훈련합니다.
    약한 학습기의 오차를 계산합니다:
    err_t = sum(w_i * (h_t(xi) != yi)) / sum(w_i), i = 1, 2, ..., n

    약한 학습기의 가중치를 계산합니다:
    alpha_t = log((1 - err_t) / err_t)

    다음 반복을 위해 가중치를 업데이트합니다:
    w_i = w_i * exp(alpha_t * (h_t(xi) != yi)), i = 1, 2, ..., n

    가중치를 정규화합니다:
    w_i = w_i / sum(w_i), i = 1, 2, ..., n

출력: 약한 학습기의 앙상블 H = {h_1, h_2, ..., h_T}, 각각은 가중치 alpha_t와 연결됩니다.
```


|                 | AdaBoosting                          | Gradient Boosting                     |
|-----------------|-------------------------------------|--------------------------------------|
| 대표 알고리즘   | Decision Stump                       | Decision Tree                        |
| 목적            | Bias 감소, Variance 증가              | Bias 감소, Variance 감소                 |
| 가중치          | 잘못 분류된 샘플에 가중치를 부여     | 잔여 오차에 대한 예측값을 부여         |
| 오차함수        | 지수함수                             | 잔여 오차 제곱 또는 절댓값              |
| 학습 방식       | 순차적으로 각 약한 학습기를 학습     | 병렬적으로 각 약한 학습기를 학습        |
| 과적합 방지     | 약한 학습기를 단순화시키는 방식 사용 | 약한 학습기를 조절하는 방식 사용         |
| 하이퍼파라미터 | 학습률, 약한 학습기 수 등           | 학습률, 약한 학습기 수, 깊이 등         |

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
df = pd.read_csv("data/hotel_bookings.csv")
df.shape

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
df.isnull().sum().describe()

In [ ]:
df.select_dtypes(include="O").head(1)

In [ ]:
lable_name = "is_canceled"
lable_name

In [ ]:
df[lable_name].value_counts()

In [ ]:
label_one_count = (df[lable_name] == 1).sum()
label_one_count

In [ ]:
df_label_not_one = df[df[lable_name] != 1].sample(label_one_count)

In [ ]:
df_under = pd.concat([df_label_not_one,  df[df[lable_name] == 1]])
df_under.shape

In [ ]:
df_under[lable_name].value_counts()

In [ ]:
X, y = df_under.drop(columns=[lable_name, "reservation_status", "reservation_status_date"]), df_under[lable_name]
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.1, random_state=42)

In [ ]:
cat_col = X.select_dtypes(exclude="number").columns
print(cat_col)
X_train[cat_col] = X_train[cat_col].astype("category")
X_test[cat_col] = X_test[cat_col].astype("category")

* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier
* https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_categorical.html

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

ct = make_column_transformer(
      (StandardScaler(),
       make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore"),
       make_column_selector(dtype_include="category")))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import make_pipeline

model_pipe = make_pipeline(ct, AdaBoostClassifier(random_state=42))
model_pipe

In [ ]:
# from sklearn.model_selection import cross_validate

# cv_result = cross_validate(model_pipe, X_test, y_test, cv=3)
# cv_result

In [ ]:
# fit & predict
model_pipe.fit(X_train, y_train)

In [ ]:
# ct.get_feature_names_out()

https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance

In [ ]:
# from sklearn.inspection import permutation_importance

# model = hist_pipe
# result = permutation_importance(model, X_train, y_train,
#                                 n_repeats=3, random_state=0)

In [ ]:
feature_names = ct.get_feature_names_out()

In [ ]:
# for i in result.importances_mean.argsort()[::-1]:
#     if result.importances_mean[i] - 2 * result.importances_std[i] > 0:
#         print(f"{feature_names[i]:<8}"
#               f"{result.importances_mean[i]:.3f}"
#               f" +/- {result.importances_std[i]:.3f}")

In [ ]:
# accuracy
model_score = model_pipe.score(X_test, y_test)
model_score

In [ ]:
y_predict = model_pipe.predict(X_test)
y_predict[:5]

In [ ]:
pd.Series(y_predict).value_counts()

In [ ]:
pd.crosstab(y_test, y_predict)